In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
# Librerias
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# Carga de datos
from google.colab import drive
drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore.csv"
reviews_file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore_user_reviews.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
google_play_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(file_path, header=True, inferSchema=True)
user_reviews_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(reviews_file_path, header=True, inferSchema=True)
google_play_data = google_play_data.rdd
user_reviews_data = user_reviews_data.rdd
# Filtra las filas en las que la categoría no sea "1.9" y crea un nuevo RDD sin esa fila
google_play_data = google_play_data.filter(lambda x: x['Category'] != "1.9")


# 15 - Calcular el promedio de rating por tipo de App.(⭐⭐)

Filtrar las filas con calificaciones no válidas (NaN) en la columna "Rating"

In [ ]:
filtered_rdd = google_play_data.filter(lambda x: str(x['Rating']) != 'nan')

Filtrar las filas con valor válido en la columna "Type"

In [ ]:
filtered_rdd = filtered_rdd.filter(lambda x: x['Type'] in ["Free", "Paid"])

Mapear el RDD para crear pares clave-valor donde la clave es el tipo de aplicación y el valor es la calificación

In [ ]:
rating_by_type = filtered_rdd.map(lambda x: (x['Type'] , float(x['Rating'])))

Reducir el RDD para calcular el promedio de calificaciones por tipo de aplicación

In [ ]:
sum_ratings_by_type = rating_by_type.reduceByKey(lambda x, y:x + y )


Ver cuantas aplicaciones hay por tipo

In [ ]:
count_by_type = rating_by_type.countByKey()

Calcular el promedio dividiendo la suma total de calificaciones por el número total de calificaciones

In [ ]:
average_ratings_by_type = sum_ratings_by_type.map(lambda x: (x[0], x[1] / count_by_type[x[0]]))

In [ ]:
average_ratings_by_type.collect()

[('Free', 4.186202546163562), ('Paid', 4.266615146831529)]